## Day 1

In [2]:
from pathlib import Path

def load_file(file_name: str) -> str:
    with Path(f"./{file_name}").open() as file:
        payload = file.read()
    return payload

def str_to_int(vec: list[str]) -> list[int]:
    return [int(n) for n in vec]

In [2]:
from collections import defaultdict

payload = load_file("d01.txt")


numbers = []
for i, line in enumerate(payload.splitlines()):
    elements = list(line)
    temp = []
    for e in elements:
        try:
            temp.append(str(int(e)))
        except ValueError as e:
            pass
    numbers.append(temp)

numbers2 = []
for line in numbers:
    first = line[0]
    last = line[-1]

    num = int("".join([first, last]))
    numbers2.append(num)

sum(numbers2)


55208

In [3]:
digits_text = {
    "one": 1,
    "two": 2,
    "three": 3,
    "four": 4,
    "five": 5,
    "six": 6,
    "seven": 7,
    "eight": 8,
    "nine": 9,
}

# payload = """
# two1nine
# eightwothree
# abcone2threexyz
# xtwone3four
# 4nineeightseven2
# zoneight234
# 7pqrstsixteen
# """
payload = load_file("d01.txt")

numbers = []
for line in payload.splitlines():
    temp = {}
    for txt, num in digits_text.items():
        if txt in line:
            index = [i for i in range(len(line)) if line.startswith(txt, i)]
            for ind in index:
                temp[ind] = digits_text[txt]
        if str(num) in line:
            index = [i for i in range(len(line)) if line.startswith(str(num), i)]
            for ind in index:
                temp[ind] = num
    numbers.append(temp)

numbers
numbers2 = []
for line in numbers:
    first = min(line.keys())
    last = max(line.keys())
    first = line[first]
    last = line[last]
    numbers2.append(int("".join([str(first), str(last)])))

sum(numbers2)

54578

## Day 2

In [4]:
# step 1
# find all possible games with: only 12 red cubes, 13 green cubes, and 14 blue cubes
# sum up their IDs

payload = load_file("d02.txt")

possible_games_set = {
    "red": 12,
    "green": 13,
    "blue": 14,
}

possible_games_solution = []
for line in payload.splitlines():
    game_id, game = line.split(":")
    game_id = game_id.split(" ")[-1]
    game_id = int(game_id)

    game = game.split(";")
    possible = []
    for sets in game:
        for role in sets.split(","):
            num, color = role.split(" ")[-2:]
            num = int(num)

            if num <= possible_games_set[color]:
                possible.append(True)
            else:
                possible.append(False)
    
    if all(possible):
        possible_games_solution.append(game_id)

sum(possible_games_solution)

3099

In [5]:
# step 2
# what is the minimum number of each color of cubes needed for each game?
# multiply them per game (called power) and that add up the games
from math import prod
from collections import defaultdict

solution = []
for line in payload.splitlines():
    _, game = line.split(":")
    game = game.split(";")

    min_set = defaultdict(int)
    for sets in game:
        for role in sets.split(","):
            num, col = role.split(" ")[-2:]
            num = int(num)

            if num >= min_set[col]:
                min_set[col] = num

    power = prod(min_set.values())
    solution.append(power)

sum(solution)

72970

## Day 3

In [9]:
# step 1
# add up all the part numbers in the engine schematic
# all numbers next to symbol are counted (period "." is not a symbol)
from string import punctuation
import re
payload = load_file("d03.txt")

SCAN_BOX = [(-1, 0), (1, 0), (0, -1), (0, 1), (-1, -1), (-1, 1), (1, -1), (1, 1)]
SYMBOLS = set(punctuation) - set(".")  

LINES = payload.splitlines()
ROWS = len(LINES)
COLS = len(LINES[0])

def check_symbol_adjacent(row: int, col: int) -> bool:
    """
    Check if symbol is adjacent to a number making it a necessary part.
    """
    return 0 <= row < ROWS \
        and 0 <= col < COLS \
        and LINES[row][col] in SYMBOLS

solution = []
for i, line in enumerate(LINES):
    indexes = re.finditer(r"\d+", line) # find digits adjacents to each other
    for idx in indexes:
        first = idx.start()
        last = idx.end() - 1

        found = []
        for d_row, d_col in SCAN_BOX:
            new_row = i + d_row
            found.append(check_symbol_adjacent(new_row, first + d_col))
            found.append(check_symbol_adjacent(new_row, last + d_col))

        if any(found):
            solution.append(int(idx.group()))

sum(solution)

525911

In [12]:
# step 2
# a gear is any * symbol that is adjacent to exactly two part numbers
# gear ratio is the result of multiplying those numbers
from collections import defaultdict

SYMBOLS = set("*")
GEARS = defaultdict(list)

LINES = payload.splitlines()
ROWS = len(LINES)
COLS = len(LINES[0])

def check_gear_adjacent(row: int, col: int) -> bool:
    if 0 <= row < ROWS and 0 <= col < COLS:
        if LINES[row][col] in SYMBOLS:
            return True
    return False


for i, line in enumerate(LINES):
    indexes = re.finditer(r"\d+", line) # find digits adjacents to each other
    for idx in indexes:
        first = idx.start()
        last = idx.end() - 1

        found = []
        for d_row, d_col in SCAN_BOX:
            new_row = i + d_row
            if check_gear_adjacent(new_row, first + d_col):
                GEARS[(new_row, first + d_col)].append(int(idx.group()))
                break
            if check_gear_adjacent(new_row, last + d_col):
                GEARS[(new_row, last + d_col)].append(int(idx.group()))
                break

sum(num[0] * num[1] for num in GEARS.values() if len(num) == 2)

75805607

## Day 4

In [9]:
# step 1
payload = load_file("d04.txt")

points = [0]+[2**n for n in range(100)]
result = []
games = payload.splitlines()
for line in games:
    all_numbers = line.split(":")[-1]
    winning_numbers = all_numbers.split(" | ")[0].split()
    winning_numbers = str_to_int(winning_numbers)
    my_numbers = all_numbers.split(" | ")[1].split()
    my_numbers = str_to_int(my_numbers)

    result = len(set(my_numbers).intersection(set(winning_numbers)))
    result.append(points[result])

sum(result)
# 21558

21558

In [10]:
# step 2
from queue import Queue

payload = load_file("d04.txt")
games = payload.splitlines()
total_games = games.copy()

result = {}
card_box = Queue()

for line in games:
    card_id, all_numbers = line.split(":")
    card_id = int(card_id[4:])
    winning_numbers = all_numbers.split(" | ")[0].split()
    my_numbers = all_numbers.split(" | ")[1].split()

    n_matches = len(set(my_numbers).intersection(set(winning_numbers)))

    result[card_id] = n_matches
    card_box.put(card_id)

ans = 0
while not card_box.empty():
    ans += 1
    card_id = card_box.get()
    for ind in range(card_id+1, card_id + result[card_id] + 1):
        card_box.put(ind)

ans
# 10_425_665

10425665

## Day 5

In [25]:
from collections import defaultdict

mapping_keys = [
    "seed-to-soil map",
    "soil-to-fertilizer map",
    "fertilizer-to-water map",
    "water-to-light map",
    "light-to-temperature map",
    "temperature-to-humidity map",
    "humidity-to-location map",
]

section_keys = [
    "seeds",
    "soil",
    "fertilizer",
    "water",
    "light",
    "temperature",
    "humidity",
    "location",
]

sections = {key: [] for key in section_keys}

In [28]:
payload = load_file("d05.txt")
# get seeds 
seeds = payload.splitlines()[0].split(":")[-1].split()
seeds = str_to_int(seeds)
sections["seeds"] = seeds

# get mappings 
mapping_index = -1
mappings = {key: [] for key in mapping_keys}
for line in payload.splitlines()[2:]:
    if len(line) > 0: 
        if line[0].isalpha():
            mapping_index += 1
            current_mapping = mapping_keys[mapping_index]
        else:
            line = str_to_int(line.split())
            mappings[current_mapping].append(line)

solution = []
for seed in seeds:
    for key, values in mappings.items():
        for rng in values:
            s1, s2, end = rng
            des_rng = range(s1, end)
            src_rng = range(s2, end)
            

1280158874 0 45923291
0 1431836695 234754481
2476778759 365219074 73714061
3997869725 4152785341 16553125
3014496893 3731980396 420804945
3435301838 2667516045 60128827
2784964719 2727644872 187996890
792043155 613341484 49447658
1444573280 2476024240 74468580
2728723659 3675739336 56241060
2704677524 4236229588 24046135
2360313001 1780050354 116465758
1326082165 226504189 118491115
3495430665 3630596320 3607732
1519041860 662789142 56785021
3973462947 3383860003 24406778
1575826881 45923291 119425025
1695251906 766775600 665061095
2667516045 3323175190 6512477
4240794960 3329687667 54172336
720792676 344995304 20223770
489053726 438933135 143852141
741016446 746305099 20470501
3499038397 4169338466 66891122
761486947 582785276 30556208
3565929519 3249598798 73576392
659636803 165348316 61155873
234754481 2221724995 254299245
2674028522 4260275723 30649002
841490813 1896516112 325208883
632905867 719574163 26730936
1166699696 1666591176 113459178
2972961609 3634204052 41535284
41950000

## Day 6

In [88]:
# step 1: racing the toy boat
# my boat's acceleration: 1mm/1s
# only using timing & accelaration how many options to beat the record?

payload = load_file("d06.txt")

def split_in_half(payload):
    half = len(payload)//2
    return payload[:half], payload[half:]

# extract times and distances
time, distance = split_in_half(payload.split())
time = list(map(int, time[1:]))
distance = list(map(int, distance[1:]))

result = 1
for t, d in zip(time, distance):
    possible = 0
    for seconds in range(1, t):
        if seconds * (t - seconds) > d:
            possible += 1
    result *= possible

result
# 1083852

1083852


In [95]:
# part 2: bad kerning aka bad spacing

payload = load_file("d06.txt")
time, distance = split_in_half(payload.split())
time = int("".join(time[1:]))
distance = int("".join(distance[1:]))

result = 0
for seconds in range(1, time):
    if seconds * (time - seconds) > distance:
        result += 1

result

23501589

## Day 7

In [74]:
from collections import Counter
from enum import IntEnum
from dataclasses import dataclass, field

payload = load_file("d07.txt")

# card order of strength: A is highest & 2 is lowest
CARD_ORDER = list(reversed(["A", "K", "Q", "J", "T", "9", "8", "7", "6", "5", "4", "3", "2"]))

# hands: 5 of kind, 4 of kind, full house, 3 of kind, 2 pairs, 1 pair, high card
# if 2 games are same hand, first higher card wins

class Hand(IntEnum):
    NO_HAND = 0
    HIGH_CARD = 1
    ONE_PAIR =  2
    TWO_PAIR = 3
    THREE_KIND = 4
    FULL_HOUSE = 5
    FOUR_KIND = 6
    FIVE_KIND = 7

@dataclass
class Game:
    cards: str # = field(compare=False)
    bid: int # = field(compare=False)
    rank_hand: Hand = field(init= False)
    # rank_card: int = field(init=False, compare=False)

    def __post_init__(self):
        self.rank_hand = check_hand(self)

    def __gt__(self, other):
        if self.rank_hand > other.rank_hand:
            return True
        elif self.rank_hand == other.rank_hand:
            for index in range(5):
                c1 = CARD_ORDER.index(self.cards[index])
                c2 = CARD_ORDER.index(other.cards[index])
                if c1 > c2:
                    return True
                elif c1 == c2:
                    continue
                else:
                    return False
        else:
            return False
 

def check_hand(game: Game) -> Hand:
    card_count = Counter(game.cards)

    if len(card_count.values()) == 1:
        return Hand.FIVE_KIND
    if set(card_count.values()) == set([1, 4]):
        return Hand.FOUR_KIND
    if set(card_count.values()) == set([2, 3]):
        return Hand.FULL_HOUSE
    if sorted(card_count.values(), reverse=True) == [3, 1, 1]:
        return Hand.THREE_KIND
    if sorted(card_count.values(), reverse=True) == [2, 2, 1]:
        return Hand.TWO_PAIR
    if sorted(card_count.values(), reverse=True) == [2, 1, 1, 1]:
        return Hand.ONE_PAIR
    if len(card_count.values()) == 5:
        return Hand.HIGH_CARD
    

games = []
for game in payload.splitlines():
    cards, bid = game.split(" ")
    games.append(
        Game(cards=cards, bid=int(bid))
    )

sorted_games = sorted(games, reverse=True)
result = 0
for game, rank in zip(sorted_games, reversed(range(1, len(games)+1))):
    result += rank * game.bid

result
# 248453531

248453531

In [62]:
CARD_ORDER = list(reversed(["A", "K", "Q", "T", "9", "8", "7", "6", "5", "4", "3", "2", "J"]))


def replace_j_most_common_card(game: Game) -> str:
    cards = game.cards.replace("J", "")
    common_card = sorted(Counter(cards).items())[0][0]
    return game.cards.replace("J", common_card)


def replace_j_highest_card(game: Game) -> str:
    cards = list(game.cards.replace("J", ""))
    max_card = max([CARD_ORDER.index(c) for c in cards])
    max_card = CARD_ORDER[max_card]
    return game.cards.replace("J", max_card)

replace_j_highest_card(Game(cards="7JJK7", bid=796))

'7KKK7'